# 1. Extract grid coordinates and fault id from Petrel exported files. Only need to do once for a 3D grid in Petrel. Save it as numpy array for later use.

# 2. Extract pressure data from CMG simulation results (rwo files).

In [24]:
from rwo2npy import rwo2npy

pres_npy = rwo2npy(
    rwo_file_path = "data/case1_PRES.rwo",
    save_dir = "results/case1_PRES.npy"
    )

Parsing file: data/case1_PRES.rwo
Processing time step: 0.0 (2030-Jan-01)
Processing time step: 3652.0 (2040-Jan-01)
Processing time step: 7305.0 (2050-Jan-01)
Processing time step: 10957.0 (2060-Jan-01)
Processing time step: 189926.0 (2550-Jan-01)
Processing time step: 372547.0 (3050-Jan-01)
Found 6 time steps
Grid dimensions: I=107, J=117, K=79, Time=6


# 3. Peform fault slip analysis

In [18]:
import numpy as np

def StressTransform3D(Pf, SH, Sh, Sv, phi, theta):
    # construct stress tensor field: shape (...,3,3)
    s = np.zeros(Pf.shape + (3,3))
    s[...,0,0] = SH - Pf
    s[...,1,1] = Sh - Pf
    s[...,2,2] = Sv - Pf
    # pre-calculate trigonometric values
    cos_phi, sin_phi = np.cos(np.radians(phi)), np.sin(np.radians(phi))
    cos_theta, sin_theta = np.cos(np.radians(theta)), np.sin(np.radians(theta))
    # perform stress tranformation
    # first rotate around z axis (vertical stress direction)
    Rz = np.array([[cos_phi,-sin_phi,0],
                   [sin_phi, cos_phi,0],
                   [0,0,1]])
    # next rotate around x axis (maximum horizotnal stress direction)
    Rx = np.array([[1,0,0],
                   [0,cos_theta,-sin_theta],
                   [0,sin_theta, cos_theta]])
    # compute rotation matrix using rotation components in reverse order
    R = Rx @ Rz
    # perform rotation
    # einsum does the batched matrix multiplication
    result = np.einsum("ab,...bc,cd->...ad", R, s, R.T)
    # retrieve stresses from matrices
    tau1 = result[...,2,0] # shear stress component in the strike direction
    tau2 = result[...,2,1] # shear stress component in the dip direction
    tau  = np.sqrt(tau1**2 + tau2**2) # shear stress
    sigma = result[...,2,2] # normal stress
    
    return sigma, tau


In [ ]:
# input stress state
SH_grad = 15.25 #maximum horizontal stress gradient in MPa/km
Sh_grad = 4.89 #minimun horizontal stress gradient in MPa/km
Sv_grad = 11.69 #vertical stress gradient in MPa/km
SH_azi = 292.13 #maximum horizontal stress direction in degree, relative to north clockwise, range [0,180]
mu = 0.6 #coefficient of friction
cohesion = 1 # fault cohesion in MPa

fault_strike = 10
fault_dip = 90

# load data
coor_fault = np.load('results/JD_Sula_2025_flow_coor&fault.npy')
pres = np.load('results/case1_PRES.npy')
# extract coordinates
x_coor = coor_fault[:,:,:,0]
y_coor = coor_fault[:,:,:,1]
z_coor = coor_fault[:,:,:,2]
# compute principal stresses
SH_stress = z_coor /1000 * SH_grad
Sh_stress = z_coor /1000 * Sh_grad
Sv_stress = z_coor /1000 * Sv_grad
# extract pressure
pres_slice = pres[:,:,:,2]/1000 # convert to MPa
# compute rotation angles
phi = SH_azi - fault_strike
theta = fault_dip
# transform principla stresses to fault planes
sigma, tau = StressTransform3D(pres_slice, SH_stress, Sh_stress, Sv_stress, phi, theta)
# compute fault slip indicator, where 1 indicates slip and 0 indicates stability
fault_slip = ((tau - cohesion) / sigma >= mu).astype(int)
# save fault slip indicator
np.save('results/fault_slip.npy', fault_slip)

In [23]:
print(sigma[20,40,60])
print(tau[20,40,60])

6.620857434092926
4.5839854638042565


In [20]:
fault_slip.shape

(107, 117, 79)

In [22]:
fault_slip[1,2,3]

0

In [13]:
print(np.min(fault_slip))
print(np.max(fault_slip))

0
1
